In [1]:
import tweepy
import pandas as pd
import numpy as np
import time
from pymongo import MongoClient
consumer_key='ZkMdJtOpKzW7QW2orY4Di0hBe'
consumer_secret='S09VL1T6PQ7XTHORLpzi8z7KBRLaOZVwzPQnf4sallDwELRYJx'
oauth_token='235498901-QokbTGAFV7IMbSNKufOu0C9jRlNTIRtcTy02kxYf'
oauth_token_secret='SsqRDpEIbbMNYFZpL39QFbdO379xC61Eq5qNgceubE7lp'

In [2]:
def tweepy_setup():
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(oauth_token, oauth_token_secret)
    api=tweepy.API(auth)
    return api

## Set up Mongo

In [3]:
MONGO_HOST = 'localhost'
MONGO_PORT = 27017
DATABASE_NAME='Portfolio'

In [25]:
client_prod = MongoClient(host=MONGO_HOST,port=MONGO_PORT,unicode_decode_error_handler='ignore')
db_prod = client_prod[DATABASE_NAME]
collection_name = "Election_BJP"
collection=db_prod[collection_name]
collection

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True, unicode_decode_error_handler='ignore'), 'Portfolio'), 'Election_BJP')

#  Create a function to start the live stream and save to Mongo

In [26]:
###TODO : Extract and save the tweet ID as well since it helps remove duplicates

#Create API object
extractor=tweepy_setup()
#override tweepy.StreamListener to add logic to on_status
class MyStreamListener(tweepy.StreamListener):
    count=0
    def on_status(self, status):
        #time.sleep(1)
        #response_body=status
        #print(status)
        location=status.user.location
        name=status.user.name
        id_tweet=status.id_str
        truncated=status.truncated
        tweet=status.text
        #print(retweet)
        created_at=status.created_at
        if truncated:
            tweet=status.extended_tweet['full_text']
        record={"id":id_tweet,"Location":location,"Name":name,"Tweet":tweet,"Created At":created_at}
        #Insert to MongoDB
        collection.insert_one(record)
        #count+=1
        #if count==500:
        #    print(record)
    def on_timeout(self):
        print ('Timeout...')
        return True 
    def on_error(self, status):
        print(status)
        if status_code == 420:
           #returning False in on_data disconnects the stream
           return False

In [27]:
myStreamListener = MyStreamListener()
myStream = tweepy.Stream(auth = extractor.auth, listener=myStreamListener)

# Start the streaming

In [ ]:
myStream.disconnect()
from urllib3.exceptions import ProtocolError
while True:
    try:
        print('Streaming BJP tweets to Mongo..')
        myStream.filter(track=['BJP4India'])
        #@INCIndia @BJP4India

    except (ProtocolError, AttributeError):
        continue

Streaming BJP tweets to Mongo..


In [15]:
myStream.disconnect()